<a href="https://colab.research.google.com/github/ShivamThapa243/Information-Retrieval/blob/main/positional_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Positional Inverted Index

In [9]:
# importing dataset from drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Check Point: Checking if the files are readable

In [10]:
# accessing the dattaset
path = "/content/drive/MyDrive/Dataset/text_files/file1.txt"
with open(path, 'r') as file:
  content = file.read()

print(content)

Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.


**Function definitions (include preprocessing functions)**

In [11]:
def preprocessing(content):
  # 1. converting the string into lowercase and removing meta-tags
  content = content.lower()
  content = metadata_remover(content)
  # 2. converting string into tokens
  token_array = tokenizer_function(content)
  # 3. removing stopwords
  token_array = stopwords_remover_funtion(token_array)
  # 4. Punctuation remover function
  token_array = punctuation_remover_function(token_array)
  # 5. converting token array to string,
  # so that it can be stored as a string in the text file
  preprocessed_tokens = stringfy_token_array(token_array)
  return preprocessed_tokens
# end of functtion


# function to convert the string into lower case and no meta data
def metadata_remover(content):
  tag = False
  quote = False
  str = ""
  for ch in content:
    if ch == '<' and not quote:
      tag = True
    elif ch == '>' and not quote:
      tag = False
    elif (ch == '"' or ch == "'") and tag:
      quote = not quote
    elif not tag:
      str = str + ch

  return str
# end of function

# function to convert string into tokens
def tokenizer_function(content):
  token_array = content.split()
  return token_array
# end of function

# funtion to remove stopwords from the token array
def stopwords_remover_funtion(token_array):
  stopwords = {'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll",
                 "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's",
                 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',
                 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is',
                 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did',
                 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at',
                 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',
                 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again',
                 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both',
                 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same',
                 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've",
                 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn',
                 "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
                 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn',
                 "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"}
  token_array = []
  for token in token_array:
    if token in stopwords:
      continue
    else:
      token_array.append(token)
  return token_array
# end of funtion

# funtion to remove punctuation
def punctuation_remover_function(token_array):
  new_token = []
  puntuation = {'.', '?', '!', ',', ';', ':', '\'', '\"', '-', '_', ')', '(', '[', ']', '{', '}', '<', '>', '=', '/', '\\', '*', '#'}
  for token in token_array:
    temp = ""
    for ch in token:
      if  ch in punctuation:
        continue
      else:
        temp += ch
    new_token.append(temp)
  return new_token
#end of function

# function to convert token array into string
def stringfy_token_array(token_array):
  token_string = ""
  for str in token_array:
    token_string += str
    token_string += " "
  return token_string
# end of function

**Preprocessing the file content and storing in a new directory**

In [12]:
import os

directory_path = "/content/drive/MyDrive/Dataset"
preprocessed_directory = "/content/drive/MyDrive/Preprocessed-Dataset"
# Create the preprocessed directory if it doesn't exist
os.makedirs(preprocessed_directory, exist_ok= True)

# List all files in the directory, excluding subdirectories
# file_list = os.listdir(directory_path)
file_list = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))] # Filter out directories

print(f"Files to process: {file_list}")

# iterating through the dataset and preprocessing each file present in the directory
for filename in file_list:
  file_path = os.path.join(directory_path, filename)
  # reading the original content of the dataset
  with open(file_path, 'r') as file:
    content = file.read()

  print("Original content : ", content)
  # preprocessing the original document
  preprocessed_content = preprocessing(content)

  print("Preprocessed Content : ", preprocessed_content, "\n")

  # writing/ storing the preprocessed_content into the newly created directory
  preprocessed_file_path = os.path.join(preprocessed_directory, filename)
  with open(preprocessed_file_path, 'w') as preprocessed_file:
    preprocessed_file.write(preprocessed_content)
#end of for loop

Files to process: []


Printing 5 Original documents before and after preprocessing

In [13]:
preprocessed_directory_path = "/content/Preprocessed-Dataset"
preprocessed_file_list = os.listdir(preprocessed_directory_path)

file_count = 1

for filename in preprocessed_file_list:
  if file_count > 5:
    break

  preprocessed_file_path = os.path.join(preprocessed_directory_path, filename)
  with open(preprocessed_file_path, 'r') as file:
    preprocessed_content = file.read()

  print(file_count, f" {filename}")
  print(preprocessed_content, "\n")
  file_count += 1